# Behavior Cloning Project

### Import Libraries

In [1]:
#keras
import keras
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.backend import clear_session
from keras.models import Model

# Helper libraries
import numpy as np
import pandas as pd
import sklearn
import math
import cv2
from PIL import Image

from sklearn.model_selection import train_test_split

import random
import csv
import matplotlib.pyplot as plt

from progressbar import *
import random

print('Keras Version: {}'.format(keras.__version__))

Using TensorFlow backend.


Keras Version: 2.2.4


In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

AttributeError: module 'tensorflow' has no attribute 'get_default_session'

### Import Data

In [ ]:
def process_sample(smpl, correction_factor):
    camera_id = random.randint(0,2)
    
    if camera_id == 2:
        correction = -1*correction_factor
    else:
        correction = camera_id*correction_factor
        
    if random.random() >= 0.5:
        return [cv2.flip(cv2.imread('data/' + smpl[camera_id]), 0), 
                float(smpl[3])+correction*(-1)]
    else:
        return [cv2.imread('data/' + smpl[camera_id]), 
                float(smpl[3])+correction]

In [ ]:
def generator(samples, batch_size=32, cf = 0.2):
    while True:
        random.shuffle(samples)
        for offset in range(0, len(samples), batch_size):
            subsmpls = samples[offset:offset+batch_size]
            images, measurements = zip(*[process_sample(smpl, cf) for smpl in subsmpls])
            X_train, y_train = (np.array(images), np.array(measurements))
            X_train, y_train = sklearn.utils.shuffle(X_train, y_train)
            yield (X_train, y_train)

In [2]:
reader = csv.reader(open('data/driving_log.csv'))
lines = [line for line in reader][1:]
train_lines, valid_lines = train_test_split(lines, test_size=0.2)

batch_size=48

NameError: name 'csv' is not defined

In [3]:
# ## Process Images With Resnet and Save
# x1 = Input(shape=(160,320,3))
# x2 = Lambda(lambda x: (x/255)-0.5, input_shape=(160,320,3))(x1)
# x3 = Cropping2D(cropping=((50,20), (0,0)))(x2)
# resnet_model = ResNet50(include_top=False, weights='imagenet', input_tensor=x3, pooling=None)
# resnet_model._make_predict_function()

# for line in progressbar(lines):
#     for camera_id in range(3):
#         img = cv2.imread('data/IMG/' + line[camera_id].split('/')[1])
#         prediction = resnet_model.predict(np.expand_dims(img, axis=0))[0]
#         Image.fromarray(prediction, 'RGB').save('data/resnet/' + line[camera_id].split('/')[1])

### Define a Model

In [31]:
def create_model():
    model = keras.Sequential()
    
    model.add(Flatten(input_shape=(160, 320, 3)))

    model.add(Dense(64, activation='relu'))
    
    model.add(Dropout(0.3))
    
    model.add(Dense(48, activation='tanh'))

    model.add(Dense(1))

    model.build()

    model.compile(optimizer=keras.optimizers.Adam(), loss='mse')
    
    return model

def create_lenet():
    #Instantiate an empty model
    model = keras.Sequential()

    # C1 Convolutional Layer
    model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(160, 320, 3), padding='same'))

    # S2 Pooling Layer
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

    # C3 Convolutional Layer
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

    # S4 Pooling Layer
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # C5 Fully Connected Convolutional Layer
    model.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
    #Flatten the CNN output so that we can connect it with fully connected layers
    model.add(Flatten())

    # FC6 Fully Connected Layer
    model.add(Dense(84, activation='tanh'))

    #Output Layer
    model.add(Dense(1))

    # Compile the model
    model.compile(loss='mse', optimizer='adam')
    
    return model

def create_transfer_learning_model(base_model):
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x) 
    x = Dense(256, activation='relu')(x) 
    predictions = Dense(1)(x) 
    model = Model(input=base_model.input, output=predictions)

    model.compile(loss='mse', optimizer=keras.optimizers.Adam())

    return model

In [26]:
def train(train, valid, model):
    history = model.fit_generator(train, 
                steps_per_epoch=math.ceil(len(train_lines)/batch_size), 
                validation_data=valid, 
                validation_steps=math.ceil(len(valid_lines)/batch_size), 
                epochs=5, verbose=1, use_multiprocessing=True)
    return [model.evaluate_generator(valid, steps=len(valid_lines)), history]

### Save Model

In [27]:
def test_suite(model, correction_factor=0.2):
    if model is None:
        print("Specify a Model to Test")
        return
    elif not isinstance(model, list):
        if not isinstance(correction_factor, list):
            correction_factor = [correction_factor]
        mse, history = zip(*[train(generator(train_lines, batch_size = batch_size, cf = factor),
                              generator(valid_lines, batch_size = batch_size, cf = factor), model)
                             for factor in progressbar(correction_factor)])
    else:
        if not isinstance(correction_factor, list):
            correction_factor = [correction_factor]
        mse, history = zip(*[zip(*[train(generator(train_lines, batch_size = batch_size, cf = factor),
                              generator(valid_lines, batch_size = batch_size, cf = factor), m)
                        for factor in progressbar(correction_factor)]) for m in model])
    return mse, history

In [32]:
clear_session()

model = create_lenet()
model.summary()

train = generator(train_lines, batch_size = batch_size, cf = 0.15)
valid = generator(valid_lines, batch_size = batch_size, cf = 0.15)

history = model.fit_generator(train, 
                steps_per_epoch=math.ceil(len(train_lines)/batch_size), 
                validation_data=valid, 
                validation_steps=math.ceil(len(valid_lines)/batch_size), 
                epochs=5, verbose=1, use_multiprocessing=True)

# correction_factors = [0.1, 0.15, 0.2, 0.25, 0.30]

# mse, history = test_suite(model, correction_factor=correction_factors)

# plt.plot(correction_factors, mse[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 160, 320, 6)       456       
_________________________________________________________________
average_pooling2d_1 (Average (None, 159, 319, 6)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 155, 315, 16)      2416      
_________________________________________________________________
average_pooling2d_2 (Average (None, 77, 157, 16)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 73, 153, 120)      48120     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1340280)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 84)                112583604 
__________

UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}}]]
	 [[loss/mul/_123]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}}]]
0 successful operations.
0 derived errors ignored.

### Output Model

In [10]:
# model.save('my_model.h5')